In [1]:
from glob import glob

from matplotlib import image
from sklearn.utils import resample

from torch.utils.data import Dataset

import numpy as np
import os
import os.path as op
import glob
import sys
import logging
import SimpleITK as sitk
from tqdm import trange, tqdm
from tqdm.contrib.concurrent import thread_map
from multiprocessing import cpu_count
import torchio as tio
import h5py as h5
import matplotlib.pyplot as plt

In [7]:
h5file = '/home/viscent/hdd/dhcp/dhcp_lores/preprocessed_h5/sub-CC00582XX14.h5'

with h5.File(h5file, 'r') as f:
    for k,v in f.items():
        print(k, v.shape)

gt_t1 (145, 145, 108)
gt_t2 (145, 145, 108)
image_t1 (145, 145, 108)
image_t2 (145, 145, 108)


In [4]:
data_dir = '/media/hdd/HCP_1200'
list_dir = glob.glob(op.join(data_dir, '*'))
list_dir.sort()
list_basenames = [op.basename(x) for x in list_dir]
list_images_t1 = [op.join(x,'unprocessed','3T','T1w_MPR1',op.basename(x)+'_3T_T1w_MPR1.nii.gz') for x in list_dir]
list_images_t2 = [op.join(x,'unprocessed','3T','T2w_SPC1',op.basename(x)+'_3T_T2w_SPC1.nii.gz') for x in list_dir]

# list_dir = list_dir[:300]
# list_basenames = list_basenames[:300]
# list_images_t1 = list_images_t1[:300]
# list_images_t2 = list_images_t2[:300]

num_samples = len(list_dir)
spacing = [1.0,1.0,1.0]
spacing = np.array(spacing)
spacing *= 8
target_shape = (108, 145, 145)
factor = spacing[2] / spacing[0]
resize_transform = tio.Resize(target_shape=target_shape)
resample_transform = tio.Resample(target=spacing)
blur_transform = tio.RandomBlur(6)
transform  = tio.Compose([resample_transform,resize_transform,blur_transform])
transform_gt = resize_transform


In [9]:
def _load(x):
    img_t1, img_t2, basename = x
    preprocessed_path = op.join(data_dir, "preprocessed")
    if op.exists(op.join(preprocessed_path, basename + '.h5')):
        with h5.File(op.join(preprocessed_path, basename + '.h5'), 'r') as f:
            if input_dual_modal:
                image = np.stack(f['image_t1'][:], f['image_t2'][:])
            else:
                image = np.expand_dims(f['image_{}'.format(input_modalities[0])][:], axis=0)
            if output_dual_modal:
                gt = np.stack(f['gt_t1'][:], f['gt_t2'][:])
            else:
                gt = np.expand_dims(f['gt_{}'.format(output_modalities[0])][:], axis=0)
        return image, gt
    image_t1 = sitk.ReadImage(img_t1)
    image_t2 = sitk.ReadImage(img_t2)
    if True:
        image_t1 = transform_gt(image_t1)
        image_t2 = transform_gt(image_t2)
    image_t1 = sitk.GetArrayFromImage(image_t1)
    image_t2 = sitk.GetArrayFromImage(image_t2)
    image_t1 = (image_t1 - np.mean(image_t1)) / np.std(image_t1)
    image_t2 = (image_t2 - np.mean(image_t2)) / np.std(image_t2)
    image_t1 = image_t1
    image_t2 = image_t2
    if not op.exists(preprocessed_path):
        os.makedirs(preprocessed_path)
    sitk.WriteImage(sitk.GetImageFromArray(image_t1), op.join(preprocessed_path, basename + '_t1.nii.gz'))
    sitk.WriteImage(sitk.GetImageFromArray(image_t2), op.join(preprocessed_path, basename + '_t2.nii.gz'))

def load(list_images_t1,list_images_t2,list_basenames):
    thread_map(_load, zip(list_images_t1, list_images_t2,list_basenames), max_workers=32, total=num_samples)



In [10]:
load(list_images_t1,list_images_t2,list_basenames)

  0%|          | 0/1113 [00:00<?, ?it/s]

/home/viscent/anaconda3/envs/bunet/lib/python3.10/site-packages/torchio/transforms/preprocessing/spatial/resize.py:70: UserWarning: Output shape (109, 145, 146) != target shape (108, 145, 145). Fixing with CropOrPad
  warnings.warn(message)
/home/viscent/anaconda3/envs/bunet/lib/python3.10/site-packages/torchio/transforms/preprocessing/spatial/resize.py:70: UserWarning: Output shape (109, 145, 145) != target shape (108, 145, 145). Fixing with CropOrPad
  warnings.warn(message)
/home/viscent/anaconda3/envs/bunet/lib/python3.10/site-packages/torchio/transforms/preprocessing/spatial/resize.py:70: UserWarning: Output shape (108, 145, 146) != target shape (108, 145, 145). Fixing with CropOrPad
  warnings.warn(message)
/home/viscent/anaconda3/envs/bunet/lib/python3.10/site-packages/torchio/transforms/preprocessing/spatial/resize.py:70: UserWarning: Output shape (108, 146, 145) != target shape (108, 145, 145). Fixing with CropOrPad
  warnings.warn(message)
/home/viscent/anaconda3/envs/bunet/l